# Install Missing Packages on Colab

In [ ]:
# Install CIOOS-Ocean-Standards package
!pip install git+git://github.com/cioos-siooc/cioos-siooc-standards.git@jessy-dev#egg=version_subpkg\&subdirectory=ocean_standards 
 

# Import Packages

In [62]:
import pandas as pd
import re

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import xmltodict
import pandas as pd
from tqdm.notebook import tqdm

from ocean_standards import cf, nerc, erddap

# Retrieve all the different vocabularies needed
This only retrieve for each vocabularies, the id and the prefered Label.

In [63]:
# Vocabularies of interest
vocab_of_interest = ['P01','P02','P06','P07','P04','P64']

# Retrieve All Vocabularies of interest
df_vocab = pd.DataFrame()
pbar = tqdm(vocab_of_interest)
for vocab in pbar:
    pbar.set_description("Retrieve %s" % vocab)
    response = nerc.get_nvs_variable_info(vocabulary=vocab)
    df_vocab = df_vocab.append(pd.DataFrame(response))


# Define BODC (P01) vocabulary terms of interest

In [90]:
# Filter the data to retrieve only specific terms 
#  Those terms can easily be matched to CF terms
search_terms = {
    "Pressure (spatial coordinate) exerted by the water body": {
        "P07": "sea_water_pressure"
    },
    "Depth (spatial coordinate) relative to water surface in the water body": {
        "P07": "depth"
    },
    "Surface elevation.*of the water body": {
        "P07": "sea_floor_depth_below_sea_surface"
    },
    "Temperature.*of the water body": {
        "P07": ["sea_water_temperature", "sea_surface_temperature"]
    },
    "Electrical conductivity.*of the water body": {
        "P07": "sea_water_electrical_conductivity"
    },
    "Salinity.*of the water body": {
        "P07": "sea_water_salinity"
    },
    "Practical salinity.*of the water body": {
        "P07": "sea_water_practical_salinity"
    },
    "Concentration of oxygen {O2 CAS 7782-44-7} per unit mass of the water body": {
        "P07": ["moles_of_oxygen_per_unit_mass_in_sea_water"]
    },
    "Concentration of oxygen {O2 CAS 7782-44-7} per unit volume of the water body": {
        "P07": [
            "volume_fraction_of_oxygen_in_sea_water",
            "mole_concentration_of_dissolved_molecular_oxygen_in_sea_water",
            "mass_concentration_of_oxygen_in_sea_water"
        ]
    },
    "Concentration of phosphate {PO43- CAS 14265-44-2} per unit mass of the water body": {
        "P07": "moles_of_phosphate_per_unit_mass_in_sea_water"
    },
    "Concentration of phosphate {PO43- CAS 14265-44-2} per unit volume of the water body": {
        "P07": [
            "mass_concentration_of_phosphate_in_sea_water",
            "mole_concentration_of_phosphate_in_sea_water"
        ]
    },
    "Concentration of silicate {SiO44- CAS 17181-37-2} per unit mass of the water body": {
        "P07": "moles_of_silicate_per_unit_mass_in_sea_water"
    },
    "Concentration of silicate {SiO44- CAS 17181-37-2} per unit volume of the water body": {
        "P07": [
            "mass_concentration_of_silicate_in_sea_water",
            "mole_concentration_of_silicate_in_sea_water"
        ]
    },
    "Concentration of nitrite {NO2- CAS 14797-65-0} per unit volume of the water body": {
        "P07": "moles_of_nitrite_per_unit_mass_in_sea_water"
    },
    "Concentration of nitrite {NO2- CAS 14797-65-0} per unit mass of the water body": {
        "P07": "mole_concentration_of_nitrite_in_sea_water"
    },
    "Concentration of nitrate+nitrite {NO3+NO2} per unit volume of the water body": {
        "P07": "mole_concentration_of_nitrate_and_nitrite_in_sea_water"
    },
    "Concentration of nitrate+nitrite {NO3+NO2} per unit mass of the water body": {
        "P07": "moles_of_nitrate_and_nitrite_per_unit_mass_in_sea_water"
    },
    "Concentration of chlorophyll-a {chl-a CAS 479-61-8} per unit volume of the water body": {
        "P07": "mass_concentration_of_chlorophyll_a_in_sea_water"
    }
}
list(search_terms.keys())

['Pressure (spatial coordinate) exerted by the water body',
 'Depth (spatial coordinate) relative to water surface in the water body',
 'Surface elevation.*of the water body',
 'Temperature.*of the water body',
 'Electrical conductivity.*of the water body',
 'Salinity.*of the water body',
 'Practical salinity.*of the water body',
 'Concentration of oxygen {O2 CAS 7782-44-7} per unit mass of the water body',
 'Concentration of oxygen {O2 CAS 7782-44-7} per unit volume of the water body',
 'Concentration of phosphate {PO43- CAS 14265-44-2} per unit mass of the water body',
 'Concentration of phosphate {PO43- CAS 14265-44-2} per unit volume of the water body',
 'Concentration of silicate {SiO44- CAS 17181-37-2} per unit mass of the water body',
 'Concentration of silicate {SiO44- CAS 17181-37-2} per unit volume of the water body',
 'Concentration of nitrite {NO2- CAS 14797-65-0} per unit volume of the water body',
 'Concentration of nitrite {NO2- CAS 14797-65-0} per unit mass of the water

In [91]:
# Filter terms retrieved for exclusively the P01 (BODC) terms matching the list above
is_matching = df_vocab['prefLabel'].str.contains('|'.join(search_terms.keys()), regex=True) & \
    df_vocab['@id'].str.contains("collection\/P01", regex=True)

df_p01 = df_vocab.loc[is_matching][['@id','prefLabel']].sort_values('prefLabel')

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


# Parse BODC prefLabels to retrieve specific information

In [92]:
# Retrieve information from P01 terms 
# Parse vocabulary, version, term and index value from term id:
df_p01[['Vocabulary','version','term','index']] = \
    df_p01['@id'].str.extract('collection/(.*?)/(.*?)/([A-Z]*?)([0-9]*?)/')

# Drop repeated terms that have different number at the end (ex: [TEMP01, TEMP02] -> [TEMP01]) 
df_p01 = df_p01.sort_values(['term','index']).drop_duplicates('term')

# From P01 prefLabel, retrieve some of the information
def parse_P01(df_P01):
    df_P01['property'] = df_P01['prefLabel'].str.extract('^(.*) of')
    df_P01['instrument-method'] = df_P01['prefLabel'].str.extract('by (.*)$')
    df_P01['calibration'] = df_P01['prefLabel'].str.extract('and ((?!.*and).*(calibration|verification).*?)(?=$| and)')[0]
    df_P01['scale'] = df_P01['prefLabel'].str.extract('(ITS-90|IPTS-68|IPTS-48|PSS-78|Practical.*salinity?)',
                                            flags=re.IGNORECASE).replace
    df_P01['scale'] = df_P01['scale'].astype(str).str.replace('Practical.*salinity','PSS-78',1).dropna()
    df_P01['size'] = df_P01['prefLabel'].str.extract('\[(.*?)\]')
    df_P01['reference'] = df_P01['prefLabel'].str.extract('\((.*(Level|datum).*?)\)',re.IGNORECASE)[0]
    return df_P01

df_p01 = parse_P01(df_p01)

# Review each P01 terms connections on NVS 

In [93]:
# For each of the terms available retrieve list of broader, equivalent and narrower 
#  terms available in different vocabularies
print('Retrieve more detailed information from each term:')

pbar = tqdm(df_p01['@id'])
df_total = pd.DataFrame()
for id in pbar:
    df_total = pd.concat((df_total,nerc.get_nvs_variable_info(id )), 
                               axis='index',
                               join='outer',
                               ignore_index=True)

Retrieve more detailed information from each term:


In [95]:
def retrieve_vocab_ids(vocab_list,vocab):
    if type(vocab_list)==list:
        return ','.join([y for y in vocab_list if vocab in y])
    else:
        return ''

for vocab in vocab_of_interest:
    for relationship in ['broader','related','narrower']:
        if relationship in df_total:
            df_total[vocab + '_' + relationship + '_id'] = df_total[relationship].apply(lambda x: retrieve_vocab_ids(x,vocab))

In [96]:
# Show result
df_p01 = df_total
df_p01

,@id,@type,http://www.w3.org/2004/02/skos/core#altLabel,http://www.w3.org/2004/02/skos/core#broader,http://www.w3.org/2004/02/skos/core#definition,http://www.w3.org/2004/02/skos/core#prefLabel,http://www.w3.org/2004/02/skos/core#related,altLabel,broader,definition-en,prefLabel-en,related,http://www.w3.org/2002/07/owl#sameAs,sameAs,http://www.w3.org/2004/02/skos/core#narrower,narrower,P01_broader_id,P01_related_id,P01_narrower_id,P02_broader_id,P02_related_id,P02_narrower_id,P06_broader_id,P06_related_id,P06_narrower_id,P07_broader_id,P07_related_id,P07_narrower_id,P04_broader_id,P04_related_id,P04_narrower_id,P64_broader_id,P64_related_id,P64_narrower_id
0,http://vocab.nerc.ac.uk/collection/P01/current...,[http://www.w3.org/2004/02/skos/core#Concept],[{'@value': 'SL_UKACD_radar'}],[{'@id': 'http://vocab.nerc.ac.uk/collection/P...,"[{'@language': 'en', '@value': 'Height of the ...","[{'@language': 'en', '@value': 'Surface elevat...",[{'@id': 'http://vocab.nerc.ac.uk/collection/S...,SL_UKACD_radar,[http://vocab.nerc.ac.uk/collection/P02/curren...,Height of the sea surface relative to the low ...,Surface elevation (UK Admiralty Chart Datum) o...,[http://vocab.nerc.ac.uk/collection/S02/curren...,NaN,NaN,NaN,NaN,,,,http://vocab.nerc.ac.uk/collection/P02/current...,,,,http://vocab.nerc.ac.uk/collection/P06/current...,,,,,,,,,,
1,http://vocab.nerc.ac.uk/collection/P01/current...,[http://www.w3.org/2004/02/skos/core#Concept],[{'@value': 'ACD_HTSL_press'}],[{'@id': 'http://vocab.nerc.ac.uk/collection/S...,"[{'@language': 'en', '@value': 'Unavailable'}]","[{'@language': 'en', '@value': 'Surface elevat...",[{'@id': 'http://vocab.nerc.ac.uk/collection/P...,ACD_HTSL_press,[http://vocab.nerc.ac.uk/collection/S26/curren...,Unavailable,Surface elevation (UK Admiralty Chart Datum) o...,[http://vocab.nerc.ac.uk/collection/P06/curren...,NaN,NaN,NaN,NaN,,,,http://vocab.nerc.ac.uk/collection/P02/current...,,,,http://vocab.nerc.ac.uk/collection/P06/current...,,,,,,,,,,
2,http://vocab.nerc.ac.uk/collection/P01/current...,[http://www.w3.org/2004/02/skos/core#Concept],[{'@value': 'SeaLvl_bubbler'}],[{'@id': 'http://vocab.nerc.ac.uk/collection/S...,"[{'@language': 'en', '@value': 'Unavailable'}]","[{'@language': 'en', '@value': 'Surface elevat...",[{'@id': 'http://vocab.nerc.ac.uk/collection/P...,SeaLvl_bubbler,[http://vocab.nerc.ac.uk/collection/S26/curren...,Unavailable,Surface elevation (unspecified datum) of the w...,[http://vocab.nerc.ac.uk/collection/P06/curren...,NaN,NaN,NaN,NaN,,,,http://vocab.nerc.ac.uk/collection/P02/current...,,,,http://vocab.nerc.ac.uk/collection/P06/current...,,,,,,,,,,
3,http://vocab.nerc.ac.uk/collection/P01/current...,[http://www.w3.org/2004/02/skos/core#Concept],[{'@value': 'HTSeaLvl'}],[{'@id': 'http://vocab.nerc.ac.uk/collection/S...,"[{'@language': 'en', '@value': 'Unavailable'}]","[{'@language': 'en', '@value': 'Surface elevat...",[{'@id': 'http://vocab.nerc.ac.uk/collection/S...,HTSeaLvl,[http://vocab.nerc.ac.uk/collection/S26/curren...,Unavailable,Surface elevation (unspecified datum) of the w...,[http://vocab.nerc.ac.uk/collection/S02/curren...,NaN,NaN,NaN,NaN,,,,http://vocab.nerc.ac.uk/collection/P02/current...,,,,http://vocab.nerc.ac.uk/collection/P06/current...,,,,,,,,,,
4,http://vocab.nerc.ac.uk/collection/P01/current...,[http://www.w3.org/2004/02/skos/core#Concept],[{'@value': 'SL_CD_press'}],[{'@id': 'http://vocab.nerc.ac.uk/collection/P...,"[{'@language': 'en', '@value': 'Unavailable'}]","[{'@language': 'en', '@value': 'Surface elevat...",[{'@id': 'http://vocab.nerc.ac.uk/collection/S...,SL_CD_press,[http://vocab.nerc.ac.uk/collection/P02/curren...,Unavailable,Surface elevation (Chart Datum) of the water b...,[http://vocab.nerc.ac.uk/collection/S02/curren...,NaN,NaN,NaN,NaN,,,,http://vocab.nerc.ac.uk/collection/P02/current...,,,,http://vocab.nerc.ac.uk/collection/P06/current...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [97]:
# Add suggested P07
df_p01['suggested_P07'] = None
for index, row in df_p01.iterrows():
    for search_key, matching in search_terms.items(): 
        if re.search(search_key, row['prefLabel-en']) and  matching['P07']:
            if type(matching['P07']) is list:
                matching['P07'] = ','.join(matching['P07'])
            df_p01.loc[index,'suggested_P07'] = matching['P07']


In [98]:
df_p01

,@id,@type,http://www.w3.org/2004/02/skos/core#altLabel,http://www.w3.org/2004/02/skos/core#broader,http://www.w3.org/2004/02/skos/core#definition,http://www.w3.org/2004/02/skos/core#prefLabel,http://www.w3.org/2004/02/skos/core#related,altLabel,broader,definition-en,prefLabel-en,related,http://www.w3.org/2002/07/owl#sameAs,sameAs,http://www.w3.org/2004/02/skos/core#narrower,narrower,P01_broader_id,P01_related_id,P01_narrower_id,P02_broader_id,P02_related_id,P02_narrower_id,P06_broader_id,P06_related_id,P06_narrower_id,P07_broader_id,P07_related_id,P07_narrower_id,P04_broader_id,P04_related_id,P04_narrower_id,P64_broader_id,P64_related_id,P64_narrower_id,suggested_P07
0,http://vocab.nerc.ac.uk/collection/P01/current...,[http://www.w3.org/2004/02/skos/core#Concept],[{'@value': 'SL_UKACD_radar'}],[{'@id': 'http://vocab.nerc.ac.uk/collection/P...,"[{'@language': 'en', '@value': 'Height of the ...","[{'@language': 'en', '@value': 'Surface elevat...",[{'@id': 'http://vocab.nerc.ac.uk/collection/S...,SL_UKACD_radar,[http://vocab.nerc.ac.uk/collection/P02/curren...,Height of the sea surface relative to the low ...,Surface elevation (UK Admiralty Chart Datum) o...,[http://vocab.nerc.ac.uk/collection/S02/curren...,NaN,NaN,NaN,NaN,,,,http://vocab.nerc.ac.uk/collection/P02/current...,,,,http://vocab.nerc.ac.uk/collection/P06/current...,,,,,,,,,,,sea_floor_depth_below_sea_surface
1,http://vocab.nerc.ac.uk/collection/P01/current...,[http://www.w3.org/2004/02/skos/core#Concept],[{'@value': 'ACD_HTSL_press'}],[{'@id': 'http://vocab.nerc.ac.uk/collection/S...,"[{'@language': 'en', '@value': 'Unavailable'}]","[{'@language': 'en', '@value': 'Surface elevat...",[{'@id': 'http://vocab.nerc.ac.uk/collection/P...,ACD_HTSL_press,[http://vocab.nerc.ac.uk/collection/S26/curren...,Unavailable,Surface elevation (UK Admiralty Chart Datum) o...,[http://vocab.nerc.ac.uk/collection/P06/curren...,NaN,NaN,NaN,NaN,,,,http://vocab.nerc.ac.uk/collection/P02/current...,,,,http://vocab.nerc.ac.uk/collection/P06/current...,,,,,,,,,,,sea_floor_depth_below_sea_surface
2,http://vocab.nerc.ac.uk/collection/P01/current...,[http://www.w3.org/2004/02/skos/core#Concept],[{'@value': 'SeaLvl_bubbler'}],[{'@id': 'http://vocab.nerc.ac.uk/collection/S...,"[{'@language': 'en', '@value': 'Unavailable'}]","[{'@language': 'en', '@value': 'Surface elevat...",[{'@id': 'http://vocab.nerc.ac.uk/collection/P...,SeaLvl_bubbler,[http://vocab.nerc.ac.uk/collection/S26/curren...,Unavailable,Surface elevation (unspecified datum) of the w...,[http://vocab.nerc.ac.uk/collection/P06/curren...,NaN,NaN,NaN,NaN,,,,http://vocab.nerc.ac.uk/collection/P02/current...,,,,http://vocab.nerc.ac.uk/collection/P06/current...,,,,,,,,,,,sea_floor_depth_below_sea_surface
3,http://vocab.nerc.ac.uk/collection/P01/current...,[http://www.w3.org/2004/02/skos/core#Concept],[{'@value': 'HTSeaLvl'}],[{'@id': 'http://vocab.nerc.ac.uk/collection/S...,"[{'@language': 'en', '@value': 'Unavailable'}]","[{'@language': 'en', '@value': 'Surface elevat...",[{'@id': 'http://vocab.nerc.ac.uk/collection/S...,HTSeaLvl,[http://vocab.nerc.ac.uk/collection/S26/curren...,Unavailable,Surface elevation (unspecified datum) of the w...,[http://vocab.nerc.ac.uk/collection/S02/curren...,NaN,NaN,NaN,NaN,,,,http://vocab.nerc.ac.uk/collection/P02/current...,,,,http://vocab.nerc.ac.uk/collection/P06/current...,,,,,,,,,,,sea_floor_depth_below_sea_surface
4,http://vocab.nerc.ac.uk/collection/P01/current...,[http://www.w3.org/2004/02/skos/core#Concept],[{'@value': 'SL_CD_press'}],[{'@id': 'http://vocab.nerc.ac.uk/collection/P...,"[{'@language': 'en', '@value': 'Unavailable'}]","[{'@language': 'en', '@value': 'Surface elevat...",[{'@id': 'http://vocab.nerc.ac.uk/collection/S...,SL_CD_press,[http://vocab.nerc.ac.uk/collection/P02/curren...,Unavailable,Surface elevation (Chart Datum) of the water b...,[http://vocab.nerc.ac.uk/collection/S02/curren...,NaN,NaN,NaN,NaN,,,,http://vocab.nerc.ac.uk/collection/P02/current...,,,,http://vocab.nerc.ac.uk/colle

# Save to a csv file

In [ ]:
df_p01.sort_values('prefLabel').to_csv('NERC_P01_vocabulary.csv')